In [1]:
import sys
import os
os.environ['CLMM_MODELING_BACKEND'] = 'ccl' # here you may choose ccl or nc (NumCosmo)
sys.path.append('/pbs/throng/lsst/users/cpayerne/CLMM/examples/support')
try: import clmm
except:
    import notebook_install
    notebook_install.install_clmm_pipeline(upgrade=False)
    import clmm
import matplotlib.pyplot as plt
import numpy as np
from numpy import random
from astropy.table import Table

clmm.__version__
import clmm.polaraveraging as pa
import clmm.galaxycluster as gc
import clmm.modeling as modeling
from clmm import Cosmology 
sys.path.append('/pbs/throng/lsst/users/cpayerne/GitForThesis/DC2Analysis')
import statistics as stat
import resampling
import mock_data as mock
from scipy.optimize import curve_fit
import iminuit
from iminuit import Minuit

### Testing Modeling

In [2]:
mock_cosmo = Cosmology(H0=70.0, Omega_dm0=0.27-0.045, Omega_b0=0.045, Omega_k0=0.0)

In [3]:
cluster_z = 0.4
cluster_m = 10**15
c = 4

In [4]:
np.random.seed(15)

In [5]:
data = mock.generate_galaxy_catalog(cluster_m, 
                                    cluster_z, 
                                    c, 
                                    mock_cosmo, 
                                    200,
                                    'chang13', 
                                    zsrc_min = cluster_z + 0.1
                                    ,shapenoise = 0.05
                                    ,ngals=10000)

"""Create GalaxyCluster object"""

cl = clmm.GalaxyCluster('cluster', 0, 0, cluster_z, data)

"""Add tangential and cross components"""

# the critical surface density is recquired then set is_deltasigma = True whatever you estimate reduced shear or deltasigma

cl.compute_tangential_and_cross_components(geometry="flat", is_deltasigma = False, cosmo = mock_cosmo)

"""Create profile"""

bin_edges = pa.make_bins( 0.1, 3.7,15, method='evenlog10width')

profile = cl.make_binned_profile("radians", "Mpc", bins=bin_edges,cosmo=mock_cosmo,include_empty_bins= True,gal_ids_in_bins=True)

/pbs/home/c/cpayerne/.conda/envs/mydesc/lib/python3.7/site-packages/numpy/ma/core.py:2832: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  order=order, subok=True, ndmin=ndmin)


In [6]:
moo = clmm.Modeling(massdef = 'mean', delta_mdef = 200, halo_profile_model = 'nfw')

moo.set_cosmo(mock_cosmo)

def predict_reduced_tangential_shear(r, logm):

    m = 10.**logm 
    
    moo.set_mass(m) 
    
    moo.set_concentration(c)
    
    deltasigma = []
    
    for i, R in enumerate(r):
        
        galist = profile['gal_id'][i]
        
        z = cl.galcat['z'][galist]
        
        surface_density = np.mean(moo.eval_reduced_shear(R, cluster_z, z))
    
        deltasigma.append(surface_density)
        
    return deltasigma

In [7]:
#def predict_reduced_tangential_shear(r, logm):
    
#    return np.array([np.mean(
#        clmm.predict_reduced_tangential_shear(
#            r_proj = r, # Radial component of the profile
#            mdelta=10**logm, # Mass of the cluster [M_sun]
#            cdelta=4, # Concentration of the cluster
#            z_cluster=cluster_z, # Redshift of the cluster
#            z_source=cl.galcat[profile['gal_id'][i]]['z'], # Redshift value of each source galaxy inside the radial bin
#            cosmo=mock_cosmo,
#            delta_mdef=200,
#            halo_profile_model='nfw'
#        )) for i, r in enumerate(profile['radius'])])

In [8]:
logm, var_logm = curve_fit(predict_reduced_tangential_shear, profile['radius'], profile['gt'], sigma = profile['gt_err'],
                          bounds = [10,17])

/pbs/home/c/cpayerne/.conda/envs/mydesc/lib/python3.7/site-packages/scipy/optimize/minpack.py:755: RuntimeWarning: divide by zero encountered in true_divide
  transform = 1.0 / sigma


ValueError: Residuals are not finite in the initial point.

In [ ]:
popt_up_diag = [logm + 3*np.sqrt(var_logm)]
popt_low_diag = [logm - 3*np.sqrt(var_logm)]
y_up_diag = predict_reduced_tangential_shear(profile['radius'], *popt_up_diag)
y_down_diag = predict_reduced_tangential_shear(profile['radius'], *popt_low_diag)

In [ ]:
plt.errorbar(profile['radius'], profile['gt'], profile['gt_err'], c='k', fmt = '.', label = 'data')
plt.plot(profile['radius'], predict_reduced_tangential_shear(profile['radius'], logm), label = 'best fit')
plt.fill_between(profile['radius'], y_up_diag, y_down_diag, color = 'b', alpha = 0.2, label = '3sigma')
plt.xscale('log')
plt.yscale('log')
plt.xlim(profile['radius'][0], profile['radius'][-1])
plt.legend()

# Using Bootstrap Covariance Matrix

In [ ]:
sys.path.append('/pbs/throng/lsst/users/cpayerne/GitForThesis/DC2Analysis')
import statistics_ as stat

In [ ]:
Stat = stat.Statistics(len(profile['radius']))
n_repetition = 500
p1, p2 = [], []
for i in range(n_repetition):
    
    cl_bootstrap = resampling.bootstrap(cl)

    """Create profile"""

    profile_ = cl_bootstrap.make_binned_profile("radians", "Mpc", bins=bin_edges,cosmo=mock_cosmo,include_empty_bins= True,gal_ids_in_bins=True)

    p1.append(profile_['gt'][6])
    p2.append(profile_['gt'][7])
    
    Stat._add_realization(profile_['gt'])

In [ ]:
plt.scatter(p1, p2)
plt.axis('equal')

In [ ]:
Stat.estimate_covariance()

In [ ]:
cov = Stat.covariance_matrix * 1./(n_repetition - 1.)
Stat.estimate_correlation()
corr = Stat.correlation_matrix

In [ ]:
plt.imshow(cov)
plt.colorbar()

In [ ]:
plt.plot(profile['radius'],cov.diagonal()/(profile['gt_err']**2))

In [ ]:
def chi2(logm):
    

    y_predict = predict_reduced_tangential_shear(profile['radius'], logm)

    delta = -(profile['gt'] - y_predict)

    inv_cov = np.linalg.inv(cov)

    chi2 = np.sum(delta * inv_cov.dot(delta))
    
    return chi2

In [ ]:
m_cov = Minuit(chi2, limit_logm = (12,16),  errordef = 1)
m_cov.migrad()
m_cov.hesse()
m_cov.minos()
chi2min_dof_cov =  chi2(m_cov.values['logm'])/(len(profile['radius'])-1)

In [ ]:
popt_up_cov = [m_cov.values['logm'] + 3*m_cov.merrors['logm'].upper]
popt_low_cov = [m_cov.values['logm'] + 3*m_cov.merrors['logm'].lower]


In [ ]:
y_up_cov = predict_reduced_tangential_shear(profile['radius'], *popt_up_cov)
y_down_cov = predict_reduced_tangential_shear(profile['radius'], *popt_low_cov)

In [ ]:
m_cov_, err_m_cov_ = 10**m_cov.values['logm'],  np.log(10) * 10**m_cov.values['logm'] * m_cov.errors['logm']
m_diag, m_err_diag = 10**logm[0],np.log(10) * 10 ** logm * np.sqrt(var_logm)

In [ ]:
plt.figure(figsize = (8,8))
plt.errorbar(profile['radius'],profile['gt'],profile['gt_err'],fmt = '.', c = 'k',elinewidth=1, capsize=2, 
             markersize= 5, 
             lolims=False, 
             uplims=False, 
             capthick=None, 
             alpha = 1, label = 'g_t (clmm)')
#plt.errorbar(profile['radius'],profile['gt'],np.sqrt(cov_.diagonal()),fmt = 'x')
plt.plot(profile['radius'], predict_reduced_tangential_shear(profile['radius'], np.log10(cluster_m)), '--k', label ='input')
plt.fill_between(profile['radius'],y_up_cov,y_down_cov , color = 'g', alpha = 0.1, label = f'Full C')
plt.fill_between(profile['radius'],y_up_diag,y_down_diag ,color = 'b', alpha = 0.1,label = f'w/ gt_err (clmm)')

#plt.plot(profile['radius'],predict_reduced_tangential_shear(profile['radius'], logm), color = 'g', label = f'Full C')
#plt.plot(profile['radius'],predict_reduced_tangential_shear(profile['radius'], np.log10(m_cov_)) ,color = 'b',label = f'Diagonal C')
#plt.xlim(profile['radius'][0],profile['radius'][-1])
#plt.plot([],[],'s',c = 'w', label = f'M_fit w/ full C = {m_cov_:1e} +/- {err_m_cov_:1e}')
#plt.plot([],[],'s',c = 'w', label = f'M_fit w/ diagonal C = {m_diag:1e} +/- {m_err_diag[0][0]:1e}')
plt.xscale('log')
plt.yscale('log')
plt.legend(frameon = False, fontsize = 15, loc = 'lower left')
plt.show()